# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions

In [3]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="./chromadb")
collection = chroma_client.get_collection("udaplay")
@tool
def retrieve_game(search: str):
    """
    Semantic search: Finds matching games in the vector DB.

    Args:
        search (str): A question about the game industry (e.g., "Games released on PS5 in 2022").

    Returns:
        list[dict]: A list where each element contains:
            - Platform: str, the platform name (e.g., "PlayStation 5")
            - Name: str, the game title
            - YearOfRelease: str or int, release year
            - Description: str, additional details about the game
    """
    results = collection.query(
        query_texts=[search],
        n_results=5
    )

    games = []
    # Combine document text + metadata into structured records
    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        games.append({
            "Platform": meta.get("Platform", "Unknown"),
            "Name": meta.get("Name", "Unknown"),
            "YearOfRelease": meta.get("YearOfRelease", "Unknown"),
            "Description": doc or meta.get("Description", "N/A"),
        })

    return games

print(retrieve_game("best game for playstation 5"))

[{'Platform': 'PlayStation 3', 'Name': 'Gran Turismo 5', 'YearOfRelease': 2010, 'Description': '[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.'}, {'Platform': 'PlayStation 5', 'Name': "Marvel's Spider-Man 2", 'YearOfRelease': 2023, 'Description': "[PlayStation 5] Marvel's Spider-Man 2 (2023) - The sequel to the acclaimed Spider-Man game, featuring both Peter Parker and Miles Morales as playable characters."}, {'Platform': 'PlayStation 4', 'Name': "Marvel's Spider-Man", 'YearOfRelease': 2018, 'Description': "[PlayStation 4] Marvel's Spider-Man (2018) - An open-world superhero game that lets players swing through New York City as Spider-Man, battling iconic villains."}, {'Platform': 'PlayStation 1', 'Name': 'Gran Turismo', 'YearOfRelease': 1997, 'Description': '[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a n

#### Evaluate Retrieval Tool

In [5]:
from pydantic import BaseModel

class EvaluationReport(BaseModel):
    useful: bool
    description: str

# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
@tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]) -> EvaluationReport:
    """
    Evaluate the relevance of retrieved documents for a given question.

    Args:
        question (str): The original question from the user.
        retrieved_docs (list[dict]): The documents retrieved from the vector database.

    Returns:
        dict: A report on the usefulness of the retrieved documents.
    """

    context_str = "\n".join([f"{doc}" for doc in retrieved_docs])
    system_msg = (
        "You are an expert evaluator for retrieval-augmented generation (RAG). "
        "Your task is to determine whether the provided retrieved documents are "
        "sufficient, relevant, and reliable to answer the user's question. "
        "Respond ONLY with a strict JSON object matching this schema:\n"
        '{ "useful": <true|false>, "description": "<detailed explanation and next actions>" }'
    )

    user_msg = (
        "Question:\n"
        f"{question}\n\n"
        "Retrieved Documents:\n"
        f"{context_str}\n\n"
        "Evaluate if these documents are enough to respond to the question. "
        "Give a detailed explanation so it is possible to take action to accept it or not. "
        "If not sufficient, suggest what to retrieve next (e.g., more recent data, specific sections, different sources). "
        "Return only JSON."
    )

    
    llm = LLM(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
    return llm.invoke(
        input=[
            SystemMessage(content=system_msg),
            UserMessage(content=user_msg),
        ],
        response_format=EvaluationReport
    )

#### Game Web Search Tool

In [6]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the web
#    args:
#    - question: a question about game industry.
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=TAVILY_API_KEY, api_base_url="https://api.tavily.com")

@tool
def game_web_search(question: str):
    """
    Tool: game_web_search
    Description:
        Semantic search: Finds most relevant results from the web using Tavily.
        This tool is optimized for game industry–related questions.

    Args:
        question (str): A natural language query about the game industry.

    Returns:
        list[dict]: A list of search results, each containing:
            - title (str): The page title
            - url (str): The source link
            - content (str): The relevant snippet or summary from the page
            - site_name (str): The website name for citation
    """
    response = tavily_client.search(query=question, search_depth="basic", max_results=5)

    results = []
    for r in response.get("results", []):
        # Extract site name from URL for better citation
        import urllib.parse
        parsed_url = urllib.parse.urlparse(r.get("url", ""))
        site_name = parsed_url.netloc.replace("www.", "") if parsed_url.netloc else "Unknown Source"
        
        results.append({
            "title": r.get("title"),
            "url": r.get("url"),
            "content": r.get("content"),
            "site_name": site_name,
        })

    return results

### Agent

In [7]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
tools = [
    retrieve_game,
    evaluate_retrieval,
    game_web_search
]
agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions = (
        "You are UdaPlay, an AI Research Agent specialized in the video game industry. "
        "You always follow a Thought → Action → Answer pattern.\n\n"

        "Execution plan:\n"
        "- Use Thought to reason about the user's question.\n"
        "- First, call the RAG tool `retrieve_game` to retrieve relevant knowledge from the vector database.\n"
        "- After using `retrieve_game`, always call the `evaluate_retrieval` tool to judge whether the retrieved documents "
        "are sufficient and useful to answer the question.\n"
        "- If `evaluate_retrieval` reports that the documents are useful, use them to answer the question.\n"
        "- If `evaluate_retrieval` reports that the documents are not useful, then use the web search tool `game_web_search`.\n"
        "- If neither tool is available, or none of them provide a confident answer, respond with: "
        "'I don't know the answer.'\n\n"

        "Rules:\n"
        "- Always use `evaluate_retrieval` immediately after `retrieve_game`.\n"
        "- Never fabricate answers.\n"
        "- Only answer once you have confident information.\n"
        "- Always clearly separate Thought, Action, and Answer.\n\n"

        "Citation Format:\n"
        "Always structure your final answer in JSON format:\n"
        "{\n"
        '    "answer": "Your detailed answer here",\n'
        '    "source": "web search" or "internal knowledge",\n'
        '    "reasoning": "Brief explanation of your reasoning process",\n'
        '    "citations": {\n'
        '        "web_sources": [["https://example.com", "Site Name"]],\n'
        '        "local_sources": ["Platform: PlayStation, Game: Title, Year: 2023"]\n'
        "    }\n"
        "}\n\n"

        "Citation Rules:\n"
        "- For web searches: Include both URL and site name in web_sources\n"
        "- For local searches: Include platform, game name, and year in local_sources\n"
        "- Always include your reasoning process\n"
        "- Specify whether answer came from web search or internal knowledge\n\n"

        f"The actions you have are the Tools: {tools}.\n"
        "Your final Answer must be structured JSON with proper citations."
    )
)

In [8]:
def log_tool_execution(messages):
    tool_calls = [msg for msg in messages if hasattr(msg, 'role') and msg.role == 'tool']
    for tool_msg in tool_calls:
        tool_name = getattr(tool_msg, 'name', 'Unknown Tool')
        print(f"Tool used: {tool_name}")
        print(f"Tool output: {getattr(tool_msg, 'content', 'N/A')[:100]}...")

def format_citations(answer_text):
    """Extract and format citations from structured JSON answer"""
    import json
    try:
        # Try to parse as JSON
        answer_data = json.loads(answer_text)
        if isinstance(answer_data, dict) and 'citations' in answer_data:
            print("\n=== CITATIONS ===")
            citations = answer_data['citations']
            
            if 'web_sources' in citations and citations['web_sources']:
                print("Web Sources:")
                for source in citations['web_sources']:
                    if isinstance(source, list) and len(source) >= 2:
                        print(f"  - {source[1]}: {source[0]}")
                    else:
                        print(f"  - {source}")
            
            if 'local_sources' in citations and citations['local_sources']:
                print("Local Sources:")
                for source in citations['local_sources']:
                    print(f"  - {source}")
            
            print(f"Source Type: {answer_data.get('source', 'Unknown')}")
            print(f"Reasoning: {answer_data.get('reasoning', 'No reasoning provided')}")
            print("================")
        
        return answer_data.get('answer', answer_text)
    except json.JSONDecodeError:
        # If not JSON, return original text
        return answer_text

In [9]:
# TODO: Invoke your agent with enhanced citation reporting
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X released for Playstation 5?

print("=== Query 1: When Pokémon Gold and Silver was released? ===")
result = agent.invoke(query="When Pokémon Gold and Silver was released?").get_final_state()
log_tool_execution(result["messages"])
answer_content = result["messages"][-1].content
formatted_answer = format_citations(answer_content)
print("ANSWER:", formatted_answer)

print("\n=== Query 2: Which one was the first 3D platformer Mario game? ===")
result = agent.invoke(query="Which one was the first 3D platformer Mario game?").get_final_state()
log_tool_execution(result["messages"])
answer_content = result["messages"][-1].content
formatted_answer = format_citations(answer_content)
print("ANSWER:", formatted_answer)

print("\n=== Query 3: Was Mortal Kombat X released for Playstation 5? ===")
result = agent.invoke(query="Was Mortal Kombat X released for Playstation 5?").get_final_state()
log_tool_execution(result["messages"])
answer_content = result["messages"][-1].content
formatted_answer = format_citations(answer_content)
print("ANSWER:", formatted_answer)

=== Query 1: When Pokémon Gold and Silver was released? ===
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Tool used: retrieve_game
Tool output: "[{'Platform': 'Game Boy Color', 'Name': 'Pok\u00e9mon Gold and Silver', 'YearOfRelease': 1999, 'Des...
Tool used: evaluate_retrieval
Tool output: "role='assistant' content='{\"useful\":true,\"description\":\"The retrieved document provides the ne...

=== CITATIONS ===
Local Sources:
  - Platform: Game Boy Color, Game: Pokémon Gold and Silver, Year: 1999
Source Type: internal knowledge
Reasoning: The retrieved document provided the exact release year for Pokémon Gold and Silver, directly answering the user's question.
ANSWER: Pokémon Gold and Silve

### (Optional) Advanced

In [10]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes